# NOAA Data something something

We first need to install load packages and set the working directory. 

In [8]:
library(tidyr)
library(dplyr)
library(lubridate)
library(ggplot2)
library(reshape2)
library(data.table)
library('RPostgreSQL')
setwd("~/Desktop/R stuff")

# Reading the Data 

Now we read the relevant csv data into R and extract the date into a workable format to later group together by month and year. 

In [11]:
raw <- read.csv("1382179.csv")

#Turn into Readable Date Format
raw$credate <- as.Date(raw$DATE)

#Turn into more easily searchable format
raw$year <- year(raw$credate)
raw$month <- month(raw$credate)

#Converting to a dataframe
raw_df <- tbl_df(raw)
attach(raw_df)
head(raw)

The following objects are masked from raw_df (pos = 3):

    AWND, AWND_ATTRIBUTES, DAPR, DAPR_ATTRIBUTES, DATE, ELEVATION,
    LATITUDE, LONGITUDE, MDPR, MDPR_ATTRIBUTES, NAME, PGTM,
    PGTM_ATTRIBUTES, PRCP, PRCP_ATTRIBUTES, SNOW, SNOW_ATTRIBUTES,
    SNWD, SNWD_ATTRIBUTES, STATION, TAVG, TAVG_ATTRIBUTES, TMAX,
    TMAX_ATTRIBUTES, TMIN, TMIN_ATTRIBUTES, TOBS, TOBS_ATTRIBUTES,
    WDF2, WDF2_ATTRIBUTES, WDF5, WDF5_ATTRIBUTES, WDMV,
    WDMV_ATTRIBUTES, WSF2, WSF2_ATTRIBUTES, WSF5, WSF5_ATTRIBUTES,
    credate, month, year

The following objects are masked from raw_df (pos = 4):

    AWND, AWND_ATTRIBUTES, DAPR, DAPR_ATTRIBUTES, DATE, ELEVATION,
    LATITUDE, LONGITUDE, MDPR, MDPR_ATTRIBUTES, NAME, PGTM,
    PGTM_ATTRIBUTES, PRCP, PRCP_ATTRIBUTES, SNOW, SNOW_ATTRIBUTES,
    SNWD, SNWD_ATTRIBUTES, STATION, TAVG, TAVG_ATTRIBUTES, TMAX,
    TMAX_ATTRIBUTES, TMIN, TMIN_ATTRIBUTES, TOBS, TOBS_ATTRIBUTES,
    WDF2, WDF2_ATTRIBUTES, WDF5, WDF5_ATTRIBUTES, WDMV,
    WDMV_ATTRIBUTES, WSF2, WS

STATION,NAME,LATITUDE,LONGITUDE,ELEVATION,DATE,AWND,AWND_ATTRIBUTES,DAPR,DAPR_ATTRIBUTES,⋯,WDF5_ATTRIBUTES,WDMV,WDMV_ATTRIBUTES,WSF2,WSF2_ATTRIBUTES,WSF5,WSF5_ATTRIBUTES,credate,year,month
US1ILCK0148,"OAK LAWN 1.9 SE, IL US",41.6936,-87.729,182.3,2012-05-01,NA,,NA,,⋯,,NA,,NA,,NA,,2012-05-01,2012,5
US1ILCK0148,"OAK LAWN 1.9 SE, IL US",41.6936,-87.729,182.3,2012-05-02,NA,,NA,,⋯,,NA,,NA,,NA,,2012-05-02,2012,5
US1ILCK0148,"OAK LAWN 1.9 SE, IL US",41.6936,-87.729,182.3,2012-05-03,NA,,NA,,⋯,,NA,,NA,,NA,,2012-05-03,2012,5
US1ILCK0148,"OAK LAWN 1.9 SE, IL US",41.6936,-87.729,182.3,2012-05-04,NA,,NA,,⋯,,NA,,NA,,NA,,2012-05-04,2012,5
US1ILCK0148,"OAK LAWN 1.9 SE, IL US",41.6936,-87.729,182.3,2012-05-05,NA,,NA,,⋯,,NA,,NA,,NA,,2012-05-05,2012,5
US1ILCK0148,"OAK LAWN 1.9 SE, IL US",41.6936,-87.729,182.3,2012-05-07,NA,,NA,,⋯,,NA,,NA,,NA,,2012-05-07,2012,5


# Grouping the Data

This is the most important piece of code, which groups the data by month and then calculates the average for temperature, wind and precipitaation data

In [ ]:
#Group multiple by station, year, month
clean_month <- raw_df %>%
  group_by(STATION,NAME,year,month) %>%
  summarize(
    precip_mo=mean(PRCP, na.rm=TRUE),
    temp_mo=mean(TAVG, na.rm=TRUE),
    temp_min_mo =mean(TMIN, na.rm=TRUE),
    temp_max_mo =mean(TMAX, na.rm=TRUE),
    wind_mo=mean(as.numeric(AWND), na.rm=TRUE),
    snow_mo=mean(SNOW, na.rm=TRUE),
    lat=mean(LATITUDE),
    long=mean(LONGITUDE),
    elevation=mean(ELEVATION)
  ) #%>%

clean_year <- raw_df %>%
  group_by(STATION,NAME,year) %>%
  summarize(
    precip_yr=mean(PRCP, na.rm=TRUE),
    temp_yr=mean(TAVG, na.rm=TRUE),
    temp_min_yr =mean(TMIN, na.rm=TRUE),
    temp_max_yr =mean(TMAX, na.rm=TRUE),
    wind_yr=mean(as.numeric(AWND), na.rm=TRUE),
    snow_yr=mean(SNOW, na.rm=TRUE),
    lat=mean(LATITUDE),
    long=mean(LONGITUDE),
    elevation=mean(ELEVATION)
  ) #%>%

# Reshaping

We need to reshape the data from long format to wide format for convenience 

In [ ]:
melted_data = melt(clean_year, id=c('STATION','NAME', 'lat', 'long', 'elevation','year'))
FinalNOAAYearly = dcast(melted_data,STATION+lat+long+elevation~year+variable)

melted_data = melt(clean, id=c('STATION','NAME', 'lat', 'long', 'elevation','year', 'month'))
FinalNOAAMonthly = dcast(melted_data,STATION+lat+long+elevation~year+month+variable)

latlon = cbind(FinalNOAAYearly[,1],FinalNOAAYearly[,2], FinalNOAAYearly[,3])

write.csv(FinalNOAAMonthly, file = "NOAA_master_monthly_final.csv")
write.csv(latlon, file = "NOAASensorsLatLon.csv")
write.csv(FinalNOAAYearly, file = "NOAA_master_yearly.csv")

# Writing to Database

Now we write the data to our server 

In [ ]:
con =dbConnect(pg,user='shiv', password=PWD,host='la2.rcc.uchicago.edu',dbname='airchicago',port='5432')
dbWriteTable(con,'monthly_averages',clean,row.names=FALSE)
dbWriteTable(con,'yearly_averages',clean_year, row.names=FALSE)
dbWriteTable(con,'raw_data_noaa', raw, row.names=FALSE)